In [1]:
%config IPCompleter.greedy=True

In [2]:
#adhere to https://github.com/tensorflow/tensorflow/blob/r0.7/tensorflow/examples/tutorials/mnist/input_data.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import weapon_data as weapons


D:\z_outsourced_programs\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Variational Autoencoder

In [3]:
weapon_data = weapons.DataSet(seed=19071991) 

num_samples = weapon_data.num_examples
input_dimension = weapon_data.num_features

In [4]:
# Functions to get variables
def weights(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

### Building the networks
#### The encoder network $q_\phi(z|x)$

The decoder network takes the input image and calculates the mean $\mu =$ `z_mu` and the log variance $\log\sigma^2 =$ `z_ls2` of the Gaussian, thus producing the latent variable z.

In [5]:
n_z = 3 #Dimension of the latent space
# Input
x = tf.placeholder(tf.float32, shape=[None, input_dimension]) #Batchsize x Number of Pixels
n_hidden_1 = 14
n_hidden_2 = 15 #one more to spot errors

# First hidden layer
W_fc1 = weights([input_dimension, n_hidden_1])
b_fc1 = bias([n_hidden_1])
h_1   = tf.nn.softplus(tf.matmul(x, W_fc1) + b_fc1)

# Second hidden layer
W_fc2 = weights([n_hidden_1, n_hidden_2]) 
b_fc2 = bias([n_hidden_2])
h_2   = tf.nn.softplus(tf.matmul(h_1, W_fc2) + b_fc2)


# Parameters for the Gaussian
z_mu = tf.add(tf.matmul(h_2, weights([n_hidden_2, n_z])), bias([n_z]))
# A little trick:
#  sigma is always > 0.
#  We don't want to enforce that the network produces only positive numbers, therefore we let 
#  the network model the parameter log(\sigma^2) $\in [\infty, \infty]$
z_ls2 = tf.add(tf.matmul(h_2, weights([n_hidden_2, n_z])), bias([n_z])) 

#### The decoder network $p_\theta(x|z)$ a.k.a. generator network

Samples from a Gaussian using the given mean and the std. The sampling is done by addding a random number ensuring that backpropagation works fine.

In [6]:
batch_size = 10 #We have to define the batch size with the current version of TensorFlow
eps = tf.random_normal((batch_size, n_z), 0, 1, dtype=tf.float32) # Adding a random number
z = tf.add(z_mu, tf.multiply(tf.sqrt(tf.exp(z_ls2)), eps))  # The sampled z

In [7]:
W_fc1_g = weights([n_z, n_hidden_1])
b_fc1_g = bias([n_hidden_1])
h_1_g   = tf.nn.softplus(tf.matmul(z, W_fc1_g) + b_fc1_g)

W_fc2_g = weights([n_hidden_1, n_hidden_2])
b_fc2_g = bias([n_hidden_2])
h_2_g   = tf.nn.softplus(tf.matmul(h_1_g, W_fc2_g) + b_fc2_g)

#x_mu = tf.add(tf.matmul(h_2_g,  weights([n_hidden_2, input_dimension])), bias([input_dimension]))
#x_ls2 = tf.add(tf.matmul(h_2_g,  weights([n_hidden_2, input_dimension])), bias([input_dimension]))
x_reconstr_mean = (tf.add(tf.matmul(h_2_g,  weights([n_hidden_2, input_dimension])), bias([input_dimension])))

#### Defining the loss function

##### The reconstruction error
We assume that the data x, is Gaussian distributed with diagnoal covariance matrix $\Sigma_{ij} = \delta_{i,j} \sigma_i^2$. The parameters of that Gaussian are determined by the encoder network. The reconstruction error for the $i-th$ example in the min-batch is given by 
$$
    \mathbb{E}_{q(z|x^{(i)})}\left( \log\left(p(x^{(i)}|z)\right)\right) 
$$
we approximate the expectation with samplinging from the distribution (eaven with $L=1$)
$$
    \mathbb{E}_{q(z|x^{(i)})}\left( \log\left(p(x^{(i)}|z)\right)\right) \approx 
    \frac{1}{L} \sum_{i=1}^L \log\left(p(x^{(i)}|z^{(i,l)})\right) \approx \log\left(p(x^{(i)}|z^{(i,l)})\right)
$$

For the simple $J-dimensional$ Gaussian, we obtain the following reconstruction error (neglegting a constant term)
$$
    -\log\left(p(x^{(i)}|z^{(i)})\right) = \sum_{j=1}^D \frac{1}{2} \log(\sigma_{x_j}^2) + \frac{(x^{(i)}_j - \mu_{x_j})^2}{2 \sigma_{x_j}^2}
$$

##### The regularisation term

$$
    -D_{\tt{KL}} \left( q(z|x^{(i)}) || p(z) \right) = \frac{1}{2} \sum_{j=1}^{J} \left(1 + \log(\sigma_{z_j}^{(i)^2}) - \mu_{z_j}^{(i)^2} - \sigma_{z_j}^{(i)^2} \right)
$$

In [8]:
def kullbackLeibler(mu, log_sigma):
    """(Gaussian) Kullback-Leibler divergence KL(q||p), per training example"""
    # (tf.Tensor, tf.Tensor) -> tf.Tensor
    with tf.name_scope("KL_divergence"):
        # = -0.5 * (1 + log(sigma**2) - mu**2 - sigma**2)
        return -0.5 * tf.reduce_sum(1 + 2 * log_sigma - mu**2 -
                                    tf.exp(2 * log_sigma), 1)

def crossEntropy(obs, actual, offset=1e-7):
    """Binary cross-entropy, per training example"""
    # (tf.Tensor, tf.Tensor, float) -> tf.Tensor
    with tf.name_scope("cross_entropy"):
        # bound by clipping to avoid nan
        obs_ = tf.clip_by_value(obs, offset, 1 - offset)
    return -tf.reduce_sum(actual * tf.log(obs_) + (1 - actual) * tf.log(1 - obs_), 1)

def l1_loss(obs, actual):
    """L1 loss (a.k.a. LAD), per training example"""
    # (tf.Tensor, tf.Tensor, float) -> tf.Tensor
    with tf.name_scope("l1_loss"):
        return tf.reduce_sum(tf.abs(obs - actual) , 1)

def l2_loss(obs, actual):
    """L2 loss (a.k.a. Euclidean / LSE), per training example"""
    # (tf.Tensor, tf.Tensor, float) -> tf.Tensor
    with tf.name_scope("l2_loss"):
        return tf.reduce_sum(tf.square(obs - actual), 1)

In [9]:
#reconstr_loss = tf.reduce_sum(0.5 * x_ls2 + (tf.square(x-x_mu)/(2.0 * tf.exp(x_ls2))), 1) #varies between implementations
#reconstr_loss = -tf.reduce_sum(x * tf.log(1e-10 + x_reconstr_mean) + (1-x) * tf.log(1e-10 + 1 - x_reconstr_mean), 1)
#l2 reconstr_loss = tf.reduce_sum(tf.square(x - x_reconstr_mean), 1)
reconstr_loss = l2_loss(x_reconstr_mean, x)
#reconstr_loss = crossEntropy(x_reconstr_mean, x)
#latent_loss = kullbackLeibler(z_mu, tf.exp(z_ls2))
latent_loss = -0.5 * tf.reduce_sum(1 + z_ls2 - tf.square(z_mu) - tf.exp(z_ls2), 1)
cost = tf.reduce_mean(reconstr_loss + latent_loss)   # average over batch

# Use ADAM optimizer
optimizer =  tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [10]:
# This takes quite some time to converge. I am courious what would happen 
# if a proper optimizer is finally implemented in TensorFlow

epochs = 5000 #Set to 0, for no training
display_step = 200

init = tf.global_variables_initializer()
#saver = tf.train.Saver()


if False:
    while False:
        sess = tf.Session()  
        sess.run(init)
        batch_xs = next_batch(batch_size)
        dd = sess.run([cost], feed_dict={x: batch_xs})
        print('Test run of cost operation in while loop results in {}'.format(dd))
        if not np.isnan(dd) and not np.isinf(dd):
            break
        else:
            sess.close()
            
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        avg_cost = 0.
        total_batch = int(num_samples / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = weapon_data.next_batch(batch_size)
            _, d, z_mean_val, z_log_sigma_sq_val = sess.run((optimizer, cost,  z_mu, z_ls2), feed_dict={x: batch_xs})
            avg_cost += d / num_samples * batch_size

        # Display logs per epoch step
        if epoch % display_step == 0:
            #save_path = saver.save(sess, "weapon_data_model/vae.ckpt") #Saves the weights (not the graph)
            #print("Model saved in file: {}".format(save_path))
            print("Epoch:"+ '%04d' % epoch + ", cost=" + "{:.9f}".format(avg_cost))
            #print ("{} {} mean sigma2 {}".format(z_mean_val.min(), z_mean_val.max(), np.mean(np.exp(z_log_sigma_sq_val))))
    
    print("Average cost after training = " + "{:.9f}".format(avg_cost))



Epoch:0000, cost=531238.496621622
Epoch:0200, cost=52035.700485642
Epoch:0400, cost=2453.456627098
Epoch:0600, cost=1906.890415501
Epoch:0800, cost=1470.416515453
Epoch:1000, cost=761.709068917
Epoch:1200, cost=471.787835198
Epoch:1400, cost=399.644694973
Epoch:1600, cost=356.897369591
Epoch:1800, cost=324.744288470
Epoch:2000, cost=298.241989548
Epoch:2200, cost=292.772221952
Epoch:2400, cost=358.749814420
Epoch:2600, cost=307.914933901
Epoch:2800, cost=249.197543376
Epoch:3000, cost=227.010790851
Epoch:3200, cost=223.909217216
Epoch:3400, cost=226.628389616
Epoch:3600, cost=183.061167743
Epoch:3800, cost=204.772950250
Epoch:4000, cost=256.768676139
Epoch:4200, cost=196.521707483
Epoch:4400, cost=200.910989401
Epoch:4600, cost=174.669599275
Epoch:4800, cost=212.098470121
Average cost after training = 194.556886828


In [11]:
'''
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "weapon_data_model/vae.ckpt")
    x_sample = next_batch(batch_size)
    x_reconstr_mean
    
    var = (x_reconstr_mean, z, z_mu, z_ls2, cost, reconstr_loss, latent_loss)
    out = sess.run(var, feed_dict={x: x_sample})
    x_reconstr_mean, z_vals, z_mu_val,z_ls2_val, cost_val, reconstr_loss_val,latent_loss_val = out
    
    print(x_reconstr_mean)
    
    #var = (x_mu, x_ls2, z, z_mu, z_ls2, cost, reconstr_loss, latent_loss)
    #out = sess.run(var, feed_dict={x: x_sample})
    #x_mu_val, x_ls2_val, z_vals, z_mu_val,z_ls2_val, cost_val, reconstr_loss_val,latent_loss_val = out
    
    #print(x_mu_val, x_ls2_val)
'''

'\nsaver = tf.train.Saver()\nwith tf.Session() as sess:\n    saver.restore(sess, "weapon_data_model/vae.ckpt")\n    x_sample = next_batch(batch_size)\n    x_reconstr_mean\n    \n    var = (x_reconstr_mean, z, z_mu, z_ls2, cost, reconstr_loss, latent_loss)\n    out = sess.run(var, feed_dict={x: x_sample})\n    x_reconstr_mean, z_vals, z_mu_val,z_ls2_val, cost_val, reconstr_loss_val,latent_loss_val = out\n    \n    print(x_reconstr_mean)\n    \n    #var = (x_mu, x_ls2, z, z_mu, z_ls2, cost, reconstr_loss, latent_loss)\n    #out = sess.run(var, feed_dict={x: x_sample})\n    #x_mu_val, x_ls2_val, z_vals, z_mu_val,z_ls2_val, cost_val, reconstr_loss_val,latent_loss_val = out\n    \n    #print(x_mu_val, x_ls2_val)\n'